In [4]:
import sys
sys.path.insert(0, '../utils')
from dataPiping import *

import numpy as np
import pandas as pd
from math import exp, fabs, sqrt, log, pi

from random import random
import datetime

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback, LambdaCallback, TensorBoard, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K
from keras.engine.topology import Layer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn_pandas import DataFrameMapper


In [5]:
seed = 42
np.random.seed(seed)

In [ ]:
class InfluenceLayer(Layer):
    """ RMTPP Influence layer:
    Computes fixed terms of RMTPP loss function (negative log-likelihood)
    
        negative log-likelihood term:

        -vt.T*hj - wt(t - tj) - bt - 1/w exp(vt.T*hj + bt) + 1/w exp(vt.T*hj + wt(t-tj) + bt)

        vt.T*hj: output of previous layer (past influence)
        wt: multiplier on current influence (t-tj)
        bt: base intensity
        
    
    """

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [23]:
def _neg_log_likelihood(timings, acc_influence):
    """ Loss function for RMTPP model

    :timings: vector (t_j, t_(j+1))
    :acc_influence: rnn output = v_t * h_j
    """
#     return K.mean(K.square(timings - acc_influence), axis=-1)
    wt = -.01
    w = .01
    bt = .1
    
    t = timings[0]
    tj = timings[1]
    return -acc_influence - wt*(t - tj) \
           - bt - 1/w*K.exp(acc_influence + bt) \
           + 1/w*K.exp(acc_influence + wt*(t - tj) + bt)


In [24]:
def getModel(lr=100):
    in_neurons = 1
    out_neurons = 1
    input_hidden_neurons = 16
    lstm_hidden_neurons = 16
    output_hidden_neurons = 1

    model = Sequential()

    # # input layer (weights W_t)
    # model.add(Dense(
    #     input_hidden_neurons,
    #     input_shape=(None, in_neurons),
    #     activation='linear'))

    # recurrent layer (weights W_h)
    model.add(LSTM(
        lstm_hidden_neurons,
        return_sequences=False,
        input_shape=(None, in_neurons), activation='relu'))

    # output layer (weights v_t)
    model.add(Dense(
        out_neurons,
        activation='linear'))

    # model.compile(loss='mse', optimizer=Adam(lr=lr))
    model.compile(loss=_neg_log_likelihood, optimizer=Adam(lr=lr))

    return model

In [25]:
model = getModel()